Created env for SportRadarProject and Activated it:
# python -m venv env
# Ativated the Activate.ps1 

Using SportsRadar API to get the required tennis data :
# pip install requests

In [ ]:
import requests

url = "https://api.sportradar.com/tennis/trial/v3/en/competitions.json?api_key=16wTj59jvrpj3B0FSffrzNPbhar1QCXTaPDmFDuC"

headers = {"accept": "application/json"}

response = requests.get(url, headers=headers)

print(response.text)

Converting the json data to Python Dictionary :

In [9]:
import json
data = json.loads(response.text)

Getting the competitions data and Category data:

For parent_id get() method is used, so that if data is not present, then default is None

In [10]:
competition_data = []
category_list=[]
for i in data['competitions']:
  competition_data.append({'comp_id': i['id'],
   'comp_name': i['name'],
   'parent_id': i.get('parent_id',None),
   'type':i['type'],
   'gender':i['gender'],
   'category_id': i['category']['id']
   })
for i in data['competitions']:
   category_list.append({'category_id': i['category']['id'],
                        'category_name': i['category']['name']
                        })


Get the data into dataframes using Pandas
# pip install pandas

In [11]:
import pandas as pd
df_comp = pd.DataFrame(competition_data)
df_cat = pd.DataFrame(category_list)

Get the first 5 lines of the dataset

In [12]:
df_comp.head()

,comp_id,comp_name,parent_id,type,gender,category_id
0,sr:competition:620,Hopman Cup,None,mixed,mixed,sr:category:181
1,sr:competition:660,World Team Cup,None,mixed,men,sr:category:3
2,sr:competition:990,ATP Challenger Tour Finals,sr:competition:6239,singles,men,sr:category:72
3,sr:competition:1207,Championship International Series,None,singles,women,sr:category:6
4,sr:competition:2100,Davis Cup,None,mixed,men,sr:category:76


To Get the number of rows and column in df_comp dataframe:

In [13]:
print(df_comp.shape)

(6016, 6)


To find if nulls are there in df_comp dataframe:

In [15]:
print(df_comp.isnull().sum())

comp_id          0
comp_name        0
parent_id      189
type             0
gender           0
category_id      0
dtype: int64


In df_comp there are nulls 180 in parent_id column out of 6007 rows
To check if "sr:competition:" exists in all the rows of comp_id

In [16]:
contains_in_all = df_comp['comp_id'].str.contains('sr:competition:').all()
print (contains_in_all)

True


To Remove the string sr:competition: from all rows of column comp_id :

In [17]:
df_comp['comp_id'] = df_comp['comp_id'].str.replace('sr:competition:', '', regex=False)

In [18]:
df_comp.head()

,comp_id,comp_name,parent_id,type,gender,category_id
0,620,Hopman Cup,None,mixed,mixed,sr:category:181
1,660,World Team Cup,None,mixed,men,sr:category:3
2,990,ATP Challenger Tour Finals,sr:competition:6239,singles,men,sr:category:72
3,1207,Championship International Series,None,singles,women,sr:category:6
4,2100,Davis Cup,None,mixed,men,sr:category:76


To check the data types of df_comp dataframe :

In [19]:
print(df_comp.dtypes)

comp_id        object
comp_name      object
parent_id      object
type           object
gender         object
category_id    object
dtype: object


In [20]:
df_comp.describe()

,comp_id,comp_name,parent_id,type,gender,category_id
count,6016,6016,5827,6016,6016,6016
unique,6016,6016,2891,4,3,18
top,46734,UTR Bucharest W02,sr:competition:2577,singles,men,sr:category:785
freq,1,1,5,3080,3441,2198


In [21]:
df_cat.head()

,category_id,category_name
0,sr:category:181,Hopman Cup
1,sr:category:3,ATP
2,sr:category:72,Challenger
3,sr:category:6,WTA
4,sr:category:76,Davis Cup


In [22]:
print(df_cat.shape)

(6016, 2)


In [23]:
print(df_cat.isnull().sum())

category_id      0
category_name    0
dtype: int64


Shows no nulls in df_cat

In [24]:
duplicates = df_comp[df_comp['comp_id'].duplicated(keep=False)]
print(duplicates)

Empty DataFrame
Columns: [comp_id, comp_name, parent_id, type, gender, category_id]
Index: []


Shows no duplicates in comp_id if df_comp

To get the list of duplicated values in category_id column :

In [25]:
duplicated_values = df_cat['category_id'][df_cat['category_id'].duplicated()].unique()
print(duplicated_values)

['sr:category:3' 'sr:category:6' 'sr:category:72' 'sr:category:785'
 'sr:category:213' 'sr:category:871' 'sr:category:79' 'sr:category:1474'
 'sr:category:1476' 'sr:category:1475' 'sr:category:2400'
 'sr:category:2517' 'sr:category:2516']


In [26]:
value_counts = df_cat['category_id'].value_counts()
print(value_counts)

category_id
sr:category:785     2198
sr:category:213     2032
sr:category:72       900
sr:category:6        253
sr:category:3        223
sr:category:871      172
sr:category:2516      80
sr:category:2517      72
sr:category:79        32
sr:category:1476      16
sr:category:1474      16
sr:category:1475      11
sr:category:2400       6
sr:category:181        1
sr:category:1012       1
sr:category:74         1
sr:category:76         1
sr:category:2414       1
Name: count, dtype: int64


In [27]:
nunique_values = df_cat['category_id'].nunique()
print(nunique_values)

18


Only 18 unique category ids are there in the dataframe df_cat. We can now remove the duplicate category_id values

In [28]:
df_cat.drop_duplicates(subset=['category_id','category_name'],inplace = True)
print(df_cat)

           category_id         category_name
0      sr:category:181            Hopman Cup
1        sr:category:3                   ATP
2       sr:category:72            Challenger
3        sr:category:6                   WTA
4       sr:category:76             Davis Cup
5       sr:category:74  Billie Jean King Cup
358    sr:category:785               ITF Men
376    sr:category:213             ITF Women
679    sr:category:871              WTA 125K
922   sr:category:1012                  IPTL
923     sr:category:79            Exhibition
1341  sr:category:1474               Juniors
1345  sr:category:1476           Wheelchairs
1349  sr:category:1475               Legends
4844  sr:category:2400   Wheelchairs Juniors
5005  sr:category:2414            United Cup
5793  sr:category:2517             UTR Women
5796  sr:category:2516               UTR Men


Since all category_id values have string "sr:category:" in common, we can remove that string to create the category_id column values

In [29]:
df_cat['category_id'] = df_cat['category_id'].str.replace('sr:category:', '', regex=False)

In [30]:
print(df_cat)

     category_id         category_name
0            181            Hopman Cup
1              3                   ATP
2             72            Challenger
3              6                   WTA
4             76             Davis Cup
5             74  Billie Jean King Cup
358          785               ITF Men
376          213             ITF Women
679          871              WTA 125K
922         1012                  IPTL
923           79            Exhibition
1341        1474               Juniors
1345        1476           Wheelchairs
1349        1475               Legends
4844        2400   Wheelchairs Juniors
5005        2414            United Cup
5793        2517             UTR Women
5796        2516               UTR Men


Reset the indexes

In [31]:
df_cat.reset_index(drop=True, inplace=True)

In [32]:
print(df_cat)

   category_id         category_name
0          181            Hopman Cup
1            3                   ATP
2           72            Challenger
3            6                   WTA
4           76             Davis Cup
5           74  Billie Jean King Cup
6          785               ITF Men
7          213             ITF Women
8          871              WTA 125K
9         1012                  IPTL
10          79            Exhibition
11        1474               Juniors
12        1476           Wheelchairs
13        1475               Legends
14        2400   Wheelchairs Juniors
15        2414            United Cup
16        2517             UTR Women
17        2516               UTR Men


Remove sr:competetion: in parent_id of df_comp dataframe

In [33]:
df_comp['parent_id'] = df_comp['parent_id'].str.replace('sr:competition:', '', regex=False)

In [34]:
print(df_comp.head())

  comp_id                          comp_name parent_id     type gender  \
0     620                         Hopman Cup      None    mixed  mixed   
1     660                     World Team Cup      None    mixed    men   
2     990         ATP Challenger Tour Finals      6239  singles    men   
3    1207  Championship International Series      None  singles  women   
4    2100                          Davis Cup      None    mixed    men   

       category_id  
0  sr:category:181  
1    sr:category:3  
2   sr:category:72  
3    sr:category:6  
4   sr:category:76  


Change the category id in df_comp dataframe removing sr:category: 

In [35]:
df_comp['category_id'] = df_comp['category_id'].str.replace('sr:category:', '', regex=False)

In [36]:
print(df_comp.head())

  comp_id                          comp_name parent_id     type gender  \
0     620                         Hopman Cup      None    mixed  mixed   
1     660                     World Team Cup      None    mixed    men   
2     990         ATP Challenger Tour Finals      6239  singles    men   
3    1207  Championship International Series      None  singles  women   
4    2100                          Davis Cup      None    mixed    men   

  category_id  
0         181  
1           3  
2          72  
3           6  
4          76  


Check the datatypes of df_comp

In [37]:
print(df_comp.dtypes)

comp_id        object
comp_name      object
parent_id      object
type           object
gender         object
category_id    object
dtype: object


Check the datatypes of df_cat

In [38]:
print(df_cat.dtypes)

category_id      object
category_name    object
dtype: object


Connecting to MySql using connector:
# pip install mysql
# pip install mysql-connector-python

In [39]:
import mysql.connector

conn_mysql = mysql.connector.connect(
    host="localhost",
    user="root",
    password="Raji"
)
cursor_mysql = conn_mysql.cursor()
print("MySQL connection established!")

MySQL connection established!


Create a database in MySql:

In [40]:
cursor_mysql.execute("CREATE DATABASE IF NOT EXISTS SportsRadar_db;")
print("MySQL database 'SportsRadar_db' created successfully!")

MySQL database 'SportsRadar_db' created successfully!


Creating a table Categories in SportsRadar_db in MySql :

In [41]:
cursor_mysql.execute("USE SportsRadar_db")  # Select database
cursor_mysql.execute("""
    CREATE TABLE IF NOT EXISTS Categories (
        category_id VARCHAR(50) PRIMARY KEY,
        category_name VARCHAR(100) NOT NULL  
       
    );
""")
conn_mysql.commit()
print("Table 'Categories' created successfully in MySQL!")

Table 'Categories' created successfully in MySQL!


Inserting data into the categories table:

In [ ]:
data_list = df_cat.values.tolist()
query = """
    INSERT INTO Categories (category_id, category_name)
    VALUES (%s, %s);
"""
cursor_mysql.executemany(query, data_list)
conn_mysql.commit()
print("Data inserted using to_list()")


Create Table Competetions 

In [43]:
cursor_mysql.execute("USE SportsRadar_db")  # Select database
cursor_mysql.execute("""
    CREATE TABLE IF NOT EXISTS Competitions (
        competition_id VARCHAR(50) PRIMARY KEY,
        competition_name VARCHAR(100) NOT NULL,
        parent_id VARCHAR(50) NULL,
        type VARCHAR(20) NOT NULL,
        gender VARCHAR(10) NOT NULL,
        category_id VARCHAR(50),
        FOREIGN KEY(category_id) REFERENCES categories(category_id)
       
    );
""")
conn_mysql.commit()
print("Table 'Competitions' created successfully in MySQL!")

Table 'Competitions' created successfully in MySQL!


Insert into Competetions table data from df_comp:

In [ ]:
data_list = df_comp.values.tolist()
query = """
    INSERT INTO Competitions(competition_id, competition_name, parent_id, type, gender, category_id)
    VALUES (%s, %s, %s, %s, %s, %s);
"""
cursor_mysql.executemany(query, data_list)
conn_mysql.commit()
print("Data inserted using to_list()")


Sql Queries:

1) 	List all competitions along with their category name

In [ ]:
cursor_mysql.execute("""
SELECT 
    c.competition_id,
    c.competition_name,
    c.parent_id,
    c.type,
    c.gender,
    cat.category_name
FROM 
    Competitions c
JOIN 
    Categories cat
ON 
    c.category_id = cat.category_id;
""")

results = cursor_mysql.fetchall()

for row in results:
    print(row)

2) 	Count the number of competitions in each category

In [113]:
cursor_mysql.execute("""
SELECT 
    cat.category_name,
    COUNT(c.competition_id) AS competition_count
FROM 
    Categories cat
LEFT JOIN 
    Competitions c
ON 
    cat.category_id = c.category_id
GROUP BY 
    cat.category_name;
""")

results = cursor_mysql.fetchall()

for row in results:
    print(row)

('IPTL', 1)
('Juniors', 16)
('Legends', 11)
('Wheelchairs', 16)
('Hopman Cup', 1)
('ITF Women', 2032)
('Wheelchairs Juniors', 6)
('United Cup', 1)
('UTR Men', 80)
('UTR Women', 72)
('ATP', 223)
('WTA', 253)
('Challenger', 900)
('Billie Jean King Cup', 1)
('Davis Cup', 1)
('ITF Men', 2198)
('Exhibition', 32)
('WTA 125K', 172)


3)	Find all competitions of type 'doubles'

In [ ]:
cursor_mysql.execute("""
SELECT 
    competition_id,
    competition_name,
    parent_id,
    type,
    gender,
    category_id
FROM 
    Competitions
WHERE 
    type = 'doubles';
""")

results = cursor_mysql.fetchall()

for row in results:
    print(row)

4)	Get competitions that belong to a specific category (e.g., ITF Men)

In [ ]:
category_to_search = "ITF Men"

cursor_mysql.execute(f"""
SELECT 
    c.competition_id,
    c.competition_name,
    c.parent_id,
    c.type,
    c.gender,
    cat.category_name
FROM 
    Competitions c
JOIN 
    Categories cat
ON 
    c.category_id = cat.category_id
WHERE 
    cat.category_name = '{category_to_search}';
""")

results = cursor_mysql.fetchall()

for row in results:
    print(row)


5)	Identify parent competitions and their sub-competitions

In [117]:
cursor_mysql.execute("""
SELECT 
    parent.competition_id AS parent_id,
    parent.competition_name AS parent_name,
    child.competition_id AS sub_compitition_id,
    child.competition_name AS sub_competition_name
FROM 
    Competitions parent
JOIN 
    Competitions child
ON 
    parent.competition_id = child.parent_id;
""")

results = cursor_mysql.fetchall()

for row in results:
    print(row)


('16976', 'ITF Romania F9, Men Singles', '16978', 'ITF Romania F9, Men Doubles')
('45363', 'UTR Boca Raton W01', '45971', 'UTR Boca Raton W03')


6)	Analyze the distribution of competition types by category

In [ ]:
cursor_mysql.execute("""
SELECT 
    cat.category_name,
    c.type,
    COUNT(c.competition_id) AS competition_count
FROM 
    Categories cat
LEFT JOIN 
    Competitions c
ON 
    cat.category_id = c.category_id
GROUP BY 
    cat.category_name,
    c.type
ORDER BY 
    cat.category_name, c.type;
""")

results = cursor_mysql.fetchall()

for row in results:
    print(row)


7)	List all competitions with no parent (top-level competitions)

In [ ]:
cursor_mysql.execute("""
SELECT 
    competition_id,
    competition_name,
    type,
    gender,
    category_id
FROM 
    Competitions
WHERE 
    parent_id IS NULL;
""")

results = cursor_mysql.fetchall()

for row in results:
    print(row)


COLLECT THE COMPLEXES DATA FROM THE API ENDPOINTS-

In [ ]:
import requests

url = "https://api.sportradar.com/tennis/trial/v3/en/complexes.json?api_key=16wTj59jvrpj3B0FSffrzNPbhar1QCXTaPDmFDuC"

headers = {"accept": "application/json"}

response = requests.get(url, headers=headers)

print(response.text)

Converting the json data to Python Dictionary :

In [47]:
import json
data = json.loads(response.text)

In [ ]:
data

Getting the Complexes data and Venues data:

In [49]:
complexes_data = []
venues_list = []

for i in data['complexes']:
    complexes_data.append({
        'complex_id': i['id'],
        'complex_name': i['name']
    })

    # Check if 'venues' exists and is a list
    if 'venues' in i and isinstance(i['venues'], list):
        for venue in i['venues']:
            venues_list.append({
                'venue_id': venue['id'],
                'venue_name': venue['name'],
                'venue_city_name': venue['city_name'],
                'venue_country_name': venue['country_name'],
                'venue_country_code': venue['country_code'],
                'venue_timezone': venue['timezone'],
                'complex_id': i['id']
            })



In [50]:
df_complexes = pd.DataFrame(complexes_data)
df_venues = pd.DataFrame(venues_list)

In [51]:
df_complexes.head()

,complex_id,complex_name
0,sr:complex:705,Nacional
1,sr:complex:1078,Estadio de la Cartuja
2,sr:complex:1495,Sibur Arena
3,sr:complex:2375,Complexo de Tenis do Jamor
4,sr:complex:4032,Shree Shiv Chhatrapati Sports Complex


Check if nulls present in df_complexes:

In [52]:
print(df_complexes.isnull().sum())

complex_id      0
complex_name    0
dtype: int64


No nulls in df_complexes.
To Check if all rows of column complex_id has 'sr:complex:'

In [53]:
contains_in_all = df_complexes['complex_id'].str.contains('sr:complex:').all()
print (contains_in_all)

True


Since all rows in complex_id column has string 'sr:complex' the string can be removed

In [54]:
df_complexes['complex_id'] = df_complexes['complex_id'].str.replace('sr:complex:', '', regex=False)

Display the dataframe df_complexes

In [55]:
df_complexes.head()

,complex_id,complex_name
0,705,Nacional
1,1078,Estadio de la Cartuja
2,1495,Sibur Arena
3,2375,Complexo de Tenis do Jamor
4,4032,Shree Shiv Chhatrapati Sports Complex


In [56]:
df_venues.head()

,venue_id,venue_name,venue_city_name,venue_country_name,venue_country_code,venue_timezone,complex_id
0,sr:venue:70045,Cancha Central,Santiago,Chile,CHL,America/Santiago,sr:complex:705
1,sr:venue:74856,Centre Court,Seville,Spain,ESP,Europe/Madrid,sr:complex:1078
2,sr:venue:74858,Court One,Seville,Spain,ESP,Europe/Madrid,sr:complex:1078
3,sr:venue:1496,COURT 1,Saint Petersburg,Russia,RUS,Europe/Moscow,sr:complex:1495
4,sr:venue:1500,CENTER COURT,Saint Petersburg,Russia,RUS,Europe/Moscow,sr:complex:1495


check if nulls present in df_venues

In [57]:
print(df_venues.isnull().sum())

venue_id              0
venue_name            0
venue_city_name       0
venue_country_name    0
venue_country_code    0
venue_timezone        0
complex_id            0
dtype: int64


No nulls in df_venues.
To Check if all rows of column venue_id has string 'sr:venue:'

In [58]:
contains_in_all = df_venues['venue_id'].str.contains('sr:venue:').all()
print (contains_in_all)

True


Since all rows in venue_id column has string 'sr:venue' the string can be removed

In [59]:
df_venues['venue_id'] = df_venues['venue_id'].str.replace('sr:venue:', '', regex=False)

In [60]:
df_venues.head()

,venue_id,venue_name,venue_city_name,venue_country_name,venue_country_code,venue_timezone,complex_id
0,70045,Cancha Central,Santiago,Chile,CHL,America/Santiago,sr:complex:705
1,74856,Centre Court,Seville,Spain,ESP,Europe/Madrid,sr:complex:1078
2,74858,Court One,Seville,Spain,ESP,Europe/Madrid,sr:complex:1078
3,1496,COURT 1,Saint Petersburg,Russia,RUS,Europe/Moscow,sr:complex:1495
4,1500,CENTER COURT,Saint Petersburg,Russia,RUS,Europe/Moscow,sr:complex:1495


To check for duplicates in venue_id in df_venues:

In [61]:
duplicates = df_venues[df_venues['venue_id'].duplicated(keep=False)]
print(duplicates)

Empty DataFrame
Columns: [venue_id, venue_name, venue_city_name, venue_country_name, venue_country_code, venue_timezone, complex_id]
Index: []


To Check if all rows of column complex_id has string 'sr:complex:' in df_venues:

In [62]:
contains_in_all = df_venues['complex_id'].str.contains('sr:complex:').all()
print (contains_in_all)

True


Since all rows in complex_id column has string 'sr:complex' the string can be removed

In [63]:
df_venues['complex_id'] = df_venues['complex_id'].str.replace('sr:complex:', '', regex=False)

In [136]:
df_venues.head()

,venue_id,venue_name,venue_city_name,venue_country_name,venue_country_code,venue_timezone,complex_id
0,70045,Cancha Central,Santiago,Chile,CHL,America/Santiago,705
1,74856,Centre Court,Seville,Spain,ESP,Europe/Madrid,1078
2,74858,Court One,Seville,Spain,ESP,Europe/Madrid,1078
3,1496,COURT 1,Saint Petersburg,Russia,RUS,Europe/Moscow,1495
4,1500,CENTER COURT,Saint Petersburg,Russia,RUS,Europe/Moscow,1495


To check if there are duplicates in complex_id in df_complexes

In [64]:
duplicates = df_complexes[df_complexes['complex_id'].duplicated(keep=False)]
print(duplicates)

Empty DataFrame
Columns: [complex_id, complex_name]
Index: []


Create Table Complexes in SportsRadar_db:

In [65]:
cursor_mysql.execute("USE SportsRadar_db")  # Select database
cursor_mysql.execute("""
    CREATE TABLE IF NOT EXISTS Complexes (
        complex_id VARCHAR(50) PRIMARY KEY,
        complex_name VARCHAR(100) NOT NULL  
       
    );
""")
conn_mysql.commit()
print("Table 'Complexes' created successfully in MySQL!")

Table 'Complexes' created successfully in MySQL!


Create Table Venues in SportsRadar_db:

In [67]:
cursor_mysql.execute("USE SportsRadar_db")  # Select database
cursor_mysql.execute("""
    CREATE TABLE IF NOT EXISTS Venues (
        venue_id VARCHAR(50) PRIMARY KEY,
        venue_name VARCHAR(100) NOT NULL,
        city_name VARCHAR(100) NOT NULL,
        country_name VARCHAR(100) NOT NULL,
        country_code CHAR(3) NOT NULL,
        time_zone VARCHAR(100) NOT NULL,
        complex_id VARCHAR(50),
        FOREIGN KEY(complex_id) REFERENCES complexes(complex_id)                      
       
    );
""")
conn_mysql.commit()
print("Table 'Venues' created successfully in MySQL!")

Table 'Venues' created successfully in MySQL!


Load data into complexes table from df_complexes

In [ ]:
data_list = df_complexes.values.tolist()
query = """
    INSERT INTO complexes(complex_id, complex_name)
    VALUES (%s, %s);
"""
cursor_mysql.executemany(query, data_list)
conn_mysql.commit()
print("Data inserted using to_list()")


Load data into venues table from df_venues

In [ ]:
data_list = df_venues.values.tolist()
query = """
    INSERT INTO venues(venue_id, venue_name,city_name,country_name,country_code,time_zone,complex_id )
    VALUES (%s, %s,%s, %s,%s, %s,%s);
"""
cursor_mysql.executemany(query, data_list)
conn_mysql.commit()
print("Data inserted using to_list()")

COLLECT THE DOUBLES COMPETITOR RANKINGS DATA FROM THE API ENDPOINTS-

In [ ]:
import requests

url = "https://api.sportradar.com/tennis/trial/v3/en/double_competitors_rankings.json?api_key=16wTj59jvrpj3B0FSffrzNPbhar1QCXTaPDmFDuC"

headers = {"accept": "application/json"}

response = requests.get(url, headers=headers)

print(response.text)

Converting the json data to Python Dictionary :

In [83]:
import json
data = json.loads(response.text)

In [85]:
rankings_list = []
competitors_list = []
for ranking in data['rankings']:
    for competitor in ranking['competitor_rankings']:
        rankings_list.append({
            'rank': competitor['rank'],
            'movement': competitor['movement'],
            'points': competitor['points'], 
            'competitions_played': competitor['competitions_played'],
            'competitor_id': competitor['competitor']['id']
        })
        competitor_info = competitor['competitor']
        competitors_list.append({
            'competitor_id': competitor_info.get('id'),
            'name': competitor_info.get('name'),
            'country': competitor_info.get('country'),
            'country_code': competitor_info.get('country_code'),
            'abbreviation': competitor_info.get('abbreviation')
            
        })
        


Check if any value is null in rankings_list and competitors_list

In [86]:
df_rankings = pd.DataFrame(rankings_list)
df_competitors = pd.DataFrame(competitors_list)

Check first few rows of df_competitors:

In [87]:
df_competitors.head()

,competitor_id,name,country,country_code,abbreviation
0,sr:competitor:49363,"Pavic, Mate",Croatia,HRV,PAV
1,sr:competitor:51836,"Arevalo-Gonzalez, Marcelo",El Salvador,SLV,ARE
2,sr:competitor:14898,"Heliovaara, Harri",Finland,FIN,HEL
3,sr:competitor:637970,"Patten, Henry",Great Britain,GBR,PAT
4,sr:competitor:87690,"Thompson, Jordan",Australia,AUS,THO


Check first few rows of df_rankings:

In [88]:
df_rankings.head()

,rank,movement,points,competitions_played,competitor_id
0,1,0,9350,22,sr:competitor:49363
1,1,0,9350,22,sr:competitor:51836
2,3,0,7590,25,sr:competitor:14898
3,4,0,7590,26,sr:competitor:637970
4,5,0,6460,20,sr:competitor:87690


Check for null values in df_competitors:

In [90]:
print(df_competitors.isnull().sum())

competitor_id     0
name              0
country           0
country_code     60
abbreviation      0
dtype: int64


country_code has 60 nulls which can be filled as follows using pycountry:

In [91]:
import pycountry

# Step 1: Build country name → ISO Alpha-3 code mapping
country_to_code = {country.name: country.alpha_3 for country in pycountry.countries}

# Step 2: Define a function to fetch country code
def get_country_code(country, current_code):
    if pd.notna(current_code):
        return current_code  # If already filled, keep it
    if country in country_to_code:
        return country_to_code[country]  # Fill using country name
    return None  # Else still None

# Step 3: Apply the function to the DataFrame
df_competitors['country_code'] = df_competitors.apply(
    lambda row: get_country_code(row['country'], row['country_code']),
    axis=1
)


Now check for nulls in df_competitors:

In [ ]:
print(df_competitors.isnull().sum())

Print country names for which the country codes are null:

In [93]:
null_country_codes = df_competitors[df_competitors['country_code'].isnull()]

# Print the country names
print(null_country_codes['country'].tolist())

['Neutral', 'Neutral', 'Neutral', 'Neutral', 'Neutral', 'Neutral', 'Neutral', 'Neutral', 'Neutral', 'Neutral', 'Neutral', 'Neutral', 'Neutral', 'Neutral', 'Neutral', 'Neutral', 'Neutral', 'Neutral', 'Neutral', 'Neutral', 'Neutral', 'Neutral', 'Neutral', 'Neutral', 'Neutral', 'Neutral', 'Neutral', 'Neutral', 'Neutral', 'Neutral', 'Neutral', 'Neutral', 'Neutral', 'Neutral', 'Neutral', 'Neutral', 'Neutral', 'Neutral', 'Neutral', 'Neutral', 'Neutral', 'Neutral', 'Neutral', 'Neutral', 'Neutral', 'Neutral', 'Neutral', 'Neutral', 'Neutral', 'Neutral', 'Neutral', 'Neutral', 'Neutral', 'Neutral', 'Neutral', 'Neutral', 'Neutral', 'Neutral', 'Neutral', 'Neutral']


Since all country is 'Neutral' we can fill 'NEU' for country_codes of these countries.
Set 'NEU' manually for all Neutral countries

In [94]:

df_competitors.loc[
    (df_competitors['country'] == 'Neutral') & (df_competitors['country_code'].isnull()),
    'country_code'
] = 'NEU'


In [95]:
print(df_competitors.isnull().sum())

competitor_id    0
name             0
country          0
country_code     0
abbreviation     0
dtype: int64


Now check for nulls in df_rankings:

In [96]:
print(df_rankings.isnull().sum())

rank                   0
movement               0
points                 0
competitions_played    0
competitor_id          0
dtype: int64


In [97]:
df_competitors.head()

,competitor_id,name,country,country_code,abbreviation
0,sr:competitor:49363,"Pavic, Mate",Croatia,HRV,PAV
1,sr:competitor:51836,"Arevalo-Gonzalez, Marcelo",El Salvador,SLV,ARE
2,sr:competitor:14898,"Heliovaara, Harri",Finland,FIN,HEL
3,sr:competitor:637970,"Patten, Henry",Great Britain,GBR,PAT
4,sr:competitor:87690,"Thompson, Jordan",Australia,AUS,THO


In competitor_id we can check if all rows have the string 'sr:competitor:' in competitor_id:

In [100]:
contains_in_all = df_competitors['competitor_id'].str.contains('sr:competitor:').all()
print (contains_in_all)

True


Since all rows have the string 'sr:competitor:' we can remove it from that competitor_id:

In [101]:
df_competitors['competitor_id'] = df_competitors['competitor_id'].str.replace('sr:competitor:', '', regex=False)

In [102]:
df_competitors.head()

,competitor_id,name,country,country_code,abbreviation
0,49363,"Pavic, Mate",Croatia,HRV,PAV
1,51836,"Arevalo-Gonzalez, Marcelo",El Salvador,SLV,ARE
2,14898,"Heliovaara, Harri",Finland,FIN,HEL
3,637970,"Patten, Henry",Great Britain,GBR,PAT
4,87690,"Thompson, Jordan",Australia,AUS,THO


Get the datatypes of df_competitors:

In [104]:
df_competitors.dtypes

competitor_id    object
name             object
country          object
country_code     object
abbreviation     object
dtype: object

Create Table competitors:

In [108]:
cursor_mysql.execute("USE SportsRadar_db")  # Select database
cursor_mysql.execute("""
    CREATE TABLE IF NOT EXISTS competitors (
        competitor_id VARCHAR(50) PRIMARY KEY,
        name VARCHAR(100) NOT NULL, 
        country VARCHAR(100) NOT NULL,
        country_code CHAR(3) NOT NULL,
        abbreviation VARCHAR(10) NOT NULL    
       
    );
""")
conn_mysql.commit()
print("Table 'competitors' created successfully in MySQL!")

Table 'competitors' created successfully in MySQL!


Insert values to competitors table from df_competitors:

In [107]:
data_list = df_competitors.values.tolist()
query = """
    INSERT INTO Competitors(competitor_id, name, country, country_code, abbreviation)
    VALUES (%s, %s, %s, %s, %s);
"""
cursor_mysql.executemany(query, data_list)
conn_mysql.commit()
print("Data inserted using to_list()")


Data inserted using to_list()


Get the datatypes of df_rankings:

In [103]:
df_rankings.dtypes

rank                    int64
movement                int64
points                  int64
competitions_played     int64
competitor_id          object
dtype: object

In [122]:
df_rankings.head()

,rank,movement,points,competitions_played,competitor_id
0,1,0,9350,22,sr:competitor:49363
1,1,0,9350,22,sr:competitor:51836
2,3,0,7590,25,sr:competitor:14898
3,4,0,7590,26,sr:competitor:637970
4,5,0,6460,20,sr:competitor:87690


Check if all values in column comepetitor_id in df_rankings has the string 'sr:competitor:'

In [123]:
contains_in_all = df_rankings['competitor_id'].str.contains('sr:competitor:').all()
print (contains_in_all)

True


Replace the string 'sr:competitor:' in df_rankings :

In [124]:
df_rankings['competitor_id'] = df_rankings['competitor_id'].str.replace('sr:competitor:', '', regex=False)

Check if the string is removed:

In [125]:
df_rankings.head()

,rank,movement,points,competitions_played,competitor_id
0,1,0,9350,22,49363
1,1,0,9350,22,51836
2,3,0,7590,25,14898
3,4,0,7590,26,637970
4,5,0,6460,20,87690


Create table competitor_rankings  :

In [128]:
cursor_mysql.execute("USE SportsRadar_db")  # Select database
cursor_mysql.execute("""
        CREATE TABLE IF NOT EXISTS competitor_rankings (
        rank_id INT AUTO_INCREMENT PRIMARY KEY,
        ranking INT NOT NULL, 
        movement INT NOT NULL,
        points INT NOT NULL,
        competetions_played INT NOT NULL,
        competitor_id VARCHAR(50),
       FOREIGN KEY(competitor_id) REFERENCES Competitors(competitor_id)        
    );
""")
conn_mysql.commit()
print("Table 'competitor_rankings' created successfully in MySQL!")

Table 'competitor_rankings' created successfully in MySQL!


Insert data into table competitor_rankings from df_rankings :

In [129]:
data_list = df_rankings[['rank', 'movement', 'points', 'competitions_played', 'competitor_id']].values.tolist()

query = """
    INSERT INTO competitor_rankings(ranking, movement, points, competetions_played, competitor_id)
    VALUES (%s, %s, %s, %s, %s);
"""

cursor_mysql.executemany(query, data_list)
conn_mysql.commit()
print("Data inserted using to_list()")


Data inserted using to_list()
